# FLOPS Analysis

In [1]:
from importlib.metadata import version

pkgs = [
    "thop",
    "torch"
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

thop version: 0.1.1-2209072238
torch version: 2.7.0


## Simple Benchmark With Fixed Batch Size

In [ ]:
import torch
from thop import profile



base_config = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True
}

model_configs = {
    "gpt-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12}
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 2
input_tensor = torch.randint(0, 50527, (batch_size, 1024)).to(device)

for size in model_configs:
    base_config.update(model_configs[size])
    
    model = GPTModel(base_config).bfloat16()  
    model.to(device)
    
    # MACS --> multiply-accumulate operations
    # MACS are counted as 2 FLOPS, one for multiplication and the other for accumulate
    macs, params = profile(model, inputs=(input_tensor,), verbose=False)
    flops = 2 * macs
    print(f"{size:18}: {flops:.1e} FLOPS")
    
    del model
    torch.cuda.empty_cache()

ImportError: attempted relative import with no known parent package